In [1]:
# !pip install chromadb

In [2]:
from dotenv import load_dotenv, find_dotenv

# read local .env file
# find_dotenv 函数通常用于搜索和定位目录树中的 .env 文件，而 load_dotenv 函数用于将 .env 文件中的变量加载到环境中。
_ = load_dotenv(find_dotenv())
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)
embeddings = OpenAIEmbeddings()

In [3]:
"""
大多数内存对象都假设只有一个输入。在本文中，我们将介绍如何给具有多个输入的链添加内存。
作为这样一条链的示例，我们将向问答链添加内存。该链将相关文档和用户问题作为输入。
"""
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
 

In [4]:
with open('./files/state_of_the_union.txt') as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union) 

In [5]:
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": i} for i in range(len(texts))])
 

In [6]:
query = "What did the president say about Justice Breyer"
docs = docsearch.similarity_search(query)
 

In [7]:
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
 

In [8]:
template = """You are a chatbot having a conversation with a human.
 
Given the following extracted parts of a long document and a question, create a final answer.
 
{context}
 
{chat_history}
Human: {human_input}
Chatbot:"""
 
prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], 
    template=template
)
memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")
chain = load_qa_chain(llm, chain_type="stuff", memory=memory, prompt=prompt)
 

In [9]:
query = "What did the president say about Justice Breyer"
chain.invoke({"input_documents": docs, "human_input": query}, return_only_outputs=True)
 

{'output_text': 'The president honored Justice Stephen Breyer for his dedication and service to the country. He described Justice Breyer as an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court.'}

In [10]:
""" output
{'output_text': 'The President honored Justice Stephen Breyer, who has dedicated his life to serving the country. He thanked him for his service and mentioned that he is a retired Justice of the United States Supreme Court.'}
"""

" output\n{'output_text': 'The President honored Justice Stephen Breyer, who has dedicated his life to serving the country. He thanked him for his service and mentioned that he is a retired Justice of the United States Supreme Court.'}\n"

In [11]:
print(chain.memory.buffer)
 

Human: What did the president say about Justice Breyer
AI: The president honored Justice Stephen Breyer for his dedication and service to the country. He described Justice Breyer as an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court.


In [11]:
""" output
Human: What did the president say about Justice Breyer
AI: The president honored Justice Stephen Breyer for his dedication and service to the country. He described Justice Breyer as an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court.
"""